# Hospital Capacity Simulator

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
 
from numpy import random
from ipywidgets import interact, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, DatePicker, interactive
from ipywidgets import FloatSlider, HBox, VBox, Output
from IPython.display import display


- Set general hospital stats
- Set patient generator stats
- Plot sample days
- Select date range and generate random admissions
- Calculate and graph hourly metrics 

### Set general stats:

In [2]:
def f(Rate): return
def b(Beds): return

In [3]:
num_beds = interactive(b, Beds=IntSlider(value=312, min=100, max=500));

unplan_adm_rate = interactive(f, Rate=FloatSlider(value=.3, min=0.0, max=1.0, step=0.05));

In [4]:
form_item_layout = Layout(display='flex', flex_flow='row', justify_content='flex-start')

form_items = [VBox([Label('Total Beds:'), num_beds])]

stats_form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='flex-start',
    width='75%'))

display(stats_form)

Box(children=(VBox(children=(Label(value='Total Beds:'), interactive(children=(IntSlider(value=312, descriptio…

### Set patient generator stats & plot sample days

In [5]:
def p(los_avg, los_std): return
def b(adm_avg_bias): return
def c(adm_std_bias): return

In [6]:
style = {'description_width': 'initial'}

In [7]:
planned_los = interactive(p, los_avg=FloatSlider(value=36.0, min=0.0, max=120.0, step=0.5), 
                 los_std=FloatSlider(value=8.0, min=0.0, max=60.0, step=0.1));

unplanned_los = interactive(p, los_avg=FloatSlider(value=48.0, min=0.0, max=120.0, step=0.5), 
                 los_std=FloatSlider(value=12.0, min=0.0, max=60.0, step=0.1));

In [8]:
plan_mean_bias = interactive(b, adm_avg_bias=FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.5,  style=style))

plan_std_bias = interactive(c, adm_std_bias=FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.1,  style=style))

unplan_mean_bias = interactive(b, adm_avg_bias=FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.5,  style=style))

unplan_std_bias = interactive(c, adm_std_bias=FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.1,  style=style))

In [9]:
plan_hrly_means = np.array([[0.2, 0.2, 0.2, 0.2, 0.2, 3.5, 9, 13, 12, 12, 10, 11, 11, 11, 11, 10, 9, 6, 3, 1, 
                             0.2, 0.2, 0.2, 0.2]]).T

plan_hrly_std = np.array([[2.5, 2.1, 1.7, 1.6, 1.4, 1.35, 1.2, 3.5, 4.9, 5.7, 5.5, 4.4, 4.6, 4.8, 4.7, 4.6, 4.3, 4.2, 4.3, 
                            5, 5.5, 4.2, 3.3, 2.9]]).T

unplan_hrly_means = np.array([2.5, 2.1 ,1.7, 1.6, 1.4, 1.35, 1.2, 3.5, 4.9, 5.7, 5.5, 4.4, 4.6, 4.8, 4.7, 4.6, 4.3, 4.2, 4.3, 5,
                         5.5, 4.2, 3.3, 2.9])

unplan_hrly_std = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.875, 1.5, 3, 2.75, 2.5, 3.25, 3.25, 3, 2.75, 3, 1.75, 2, 1, 0.75, 0.25, 
                        0.05, 0.05, 0.05, 0.05])

In [10]:
def gen_day_normal(hrly_means, hrly_std, bias, rate=None):
    if not rate: rate=1
    patients = [int(random.normal(loc=m.item(), scale=s.item())*rate) for m,s in zip(hrly_means + bias, hrly_std)]
    return [p if p>=0 else 0 for p in patients]

In [11]:
def gen_day(hrly_means, bias, rate=None):
    if rate:
        return [int(random.poisson(lam=m.item())*rate) for m in hrly_means + bias]
    else: 
        return [random.poisson(lam=m.item()) for m in hrly_means + bias]

In [12]:
bias_form_items = [
                    VBox([Label('Unplanned Admission Rate (%):'), unplan_adm_rate]),
                    HBox([VBox([Label('Planned:'), planned_los, plan_mean_bias, plan_std_bias]),
                    VBox([Label('Unplanned:'), unplanned_los, unplan_mean_bias, unplan_std_bias])])]

bias_form = Box(bias_form_items, 
                layout=Layout(display='flex-grow',
                              flex_flow='column',
                              border='solid 1px',
                              align_items='flex-start',
                              justify_content='space-between',
                              width='75%'))

In [13]:
display(bias_form)

Box(children=(VBox(children=(Label(value='Unplanned Admission Rate (%):'), interactive(children=(FloatSlider(v…

In [14]:
def plot_one_day():
    for i in range(1):
        ax = sns.barplot(x=np.arange(0,24,1), y=gen_day_normal(plan_hrly_means, 
                                                               plan_hrly_std, 
                                                               plan_mean_bias.kwargs['adm_avg_bias']));
        ax.set(xlabel='Hour of the Day', ylabel='# of Patients')
        ax.set_title("Random Planned Admissions per Hour")
        plt.show();
    
    for i in range(1):
        ax = sns.barplot(x=np.arange(0,24,1), 
                         y=gen_day_normal(unplan_hrly_means, 
                                          unplan_hrly_std, 
                                          unplan_mean_bias.kwargs['adm_avg_bias'], 
                                          unplan_adm_rate.kwargs['Rate']));
        ax.set(xlabel='Hour of the Day', ylabel='# of Patients')
        ax.set_title("Random Unplanned Admissions per Hour")
        plt.show();

In [15]:
button = Button(description="Plot Sample Days")
output = Output()

In [16]:
def on_button_clicked(b):
    output.clear_output()
    with output:
        plot_one_day()

button.on_click(on_button_clicked)

In [17]:
display(button, output)

Button(description='Plot Sample Days', style=ButtonStyle())

Output()

### Plot admissions and hrly calculations for select dates

In [18]:
today = datetime.date.today()
start = DatePicker( description='Begin:', disabled=False)
start.value = today-datetime.timedelta(days=7)
end = DatePicker(description='End:', disabled=False)
end.value = today

In [19]:
dates_form = Box([HBox([start, end])], 
                 layout=Layout(display='flex', 
                      flex_flow='row',
                      border='solid 1px',
                      height='75px',
                      align_items='center', 
                      width='75%'))

In [20]:
def count_days(start, end):
    if start.value and end.value:
        delta = end.value - start.value
        return delta.days
    
def autolabel(ax, rects, xpos='center'):
    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

In [21]:
def gen_random_admissions(num_days):
    tot_planned, tot_unplanned = [],[]
    hrly_pln, hrly_unpln = [],[]
    
    for day in range(num_days):
        rand_plan = gen_day(plan_hrly_means, plan_mean_bias.kwargs['adm_avg_bias'])
        rand_unplan = gen_day(unplan_hrly_means, unplan_mean_bias.kwargs['adm_avg_bias'], unplan_adm_rate.kwargs['Rate'])
        
        hrly_pln.append(rand_plan)
        hrly_unpln.append(rand_unplan)
        tot_unplanned.append(sum(rand_unplan))
        tot_planned.append(sum(rand_plan))
    
    return tot_planned, tot_unplanned, hrly_pln, hrly_unpln

In [22]:
def get_random_admission(start_date, end_date):
    num_days = count_days(start_date, end_date)
    string_dates = [(start_date.value + datetime.timedelta(days=i)).isoformat() for i in range(0,num_days)]
    tot_planned, tot_unplanned, hrly_pl, hrly_upl = gen_random_admissions(num_days)
    return string_dates, tot_planned, tot_unplanned, hrly_pl, hrly_upl

In [23]:
def plot_admissions(string_dates, tot_planned, tot_unplanned):
    
    ind = np.arange(len(tot_planned))  # the x locations for the groups
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(12, 7))
    rects1 = ax.bar(ind - width/2, tot_planned, width, label='Planned')
    rects2 = ax.bar(ind + width/2, tot_unplanned, width, label='Unplanned')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('# of patients')
    ax.set_title('Admissions Per Day')
    ax.set_xticks(ind)
    ax.set_xticklabels(string_dates)
    fig.autofmt_xdate()
    ax.legend()

    autolabel(ax, rects1, "left")
    autolabel(ax, rects2, "right")
    
    plt.show()

In [24]:
def gen_hrly_los(tot_p, tot_up):
    p_los = [[int(random.normal(loc=planned_los.kwargs['los_avg'], scale=planned_los.kwargs['los_std'])) for x in range(a)] for a in tot_p]
    up_los = [[int(random.normal(loc=unplanned_los.kwargs['los_avg'], scale=unplanned_los.kwargs['los_std'])) for x in range(a)] for a in tot_p]
    return p_los, up_los

In [25]:
string_dates, tot_pl, tot_unpl, hrly_pl, hrly_upl = get_random_admission(start, end)
p_los, up_los = gen_hrly_los(tot_pl, tot_unpl)

In [27]:
def adm_dis_times(los):
    admit = [random.choice(np.arange(0,23,1), 1).item() for i in range(len(los))]

In [28]:
# adm_dis_times(p_los)

In [29]:
gen_button = Button(description="Generate")
output2 = Output()

In [30]:
def on_change(b):
    output2.clear_output()
    with output2:
        string_dates, tot_pl, tot_unpl, hrly_pl, hrly_upl = get_random_admission(start, end)
        plot_admissions(string_dates, tot_pl, tot_unpl)
        
unplan_adm_rate.observe(on_change)
planned_los.observe(on_change)
unplanned_los.observe(on_change)

unplan_mean_bias.observe(on_change)
unplan_std_bias.observe(on_change)
plan_mean_bias.observe(on_change)
plan_std_bias.observe(on_change)


gen_button.on_click(on_change)

start.observe(on_change)
end.observe(on_change)

In [31]:
display(dates_form)

Box(children=(HBox(children=(DatePicker(value=datetime.date(2020, 8, 31), description='Begin:'), DatePicker(va…

In [32]:
display(gen_button, output2)

Button(description='Generate', style=ButtonStyle())

Output()

In [33]:
# import networkx as nx

# def random_lobster(n, m, k, p):
#     return nx.random_lobster(n, p, p / m)

# def powerlaw_cluster(n, m, k, p):
#     return nx.powerlaw_cluster_graph(n, m, p)

# def erdos_renyi(n, m, k, p):
#     return nx.erdos_renyi_graph(n, p)

# def newman_watts_strogatz(n, m, k, p):
#     return nx.newman_watts_strogatz_graph(n, k, p)

# def plot_random_graph(n, m, k, p, generator):
#     g = generator(n, m, k, p)
#     nx.draw(g)
#     plt.show()

# interact(plot_random_graph, n=(2,30), m=(1,10), k=(1,10), p=(0.0, 1.0, 0.001),
#          generator={
#              'lobster': random_lobster,
#              'power law': powerlaw_cluster,
#              'Newman-Watts-Strogatz': newman_watts_strogatz,
#              u'Erdős-Rényi': erdos_renyi,
#          });